# Formulação de Rodrigues

**Variáveis de decisão:**
* $x_v,_c$ =
  \begin{cases}
  1, \text{se o vértice $v$ recebe a cor $c$}\\
  0, \text{caso contrário}\\
  \end{cases}

* $z_c$ =
  \begin{cases}
  1, \text{se a cor $c$ é usada}\\
  0, \text{caso contrário}\\
  \end{cases}

**Restrições**:

* Restrição de maximização

$$\Gamma_R = \max \sum_{c \in C} z_c$$

* Restrição de garatia de coloração única

$$s.a. \quad \sum_{c \in C} x_{v,c} = 1 \quad \forall v \in V$$

* Restrição de garantia que a coloração obtida será própria

$$x_{v,c} + x_{u,c} \leq z_c \quad \forall \{u,v\} \in E(G), \forall c \in C$$

* Restrição de garantia que uma cor $k$ só pode ser usada se algum vértice $v$ recebeu a cor
$k$.

$$x_c \leq \sum_{v\in V} x_{v,c}  \quad  \forall c \in C$$

* Restrição de garantia que a propriedade grundy.

$$x_{v,c} \geq 1 - \sum_{d=1}^{c-1} x_{v,d} -\sum_{u\in N(v)} x_{u,c}  \quad  \forall v\in V(G), c\in C$$

* Restrição que etabelece que os vértices isolados devem receber a
primeira cor.

$$x_{v,1}=1  	\quad	\forall v\in V(G) | N(v)=0$$

* Restrição que assegura que as primeiras cores serão usadas.

$$x_c < x_{c-1}  	\quad	\forall c\in {2,...,|C|}$$

* 

$$x_{v,c}\in {0,1}  	\quad	\forall v\in V(G), c\in C$$

*

$$x_c \geq 0  	\quad	\forall c \in C$$



In [1]:
# Instalação de pacotes necessários para execução do código
%pip install ortools
%pip install networkx

Note: you may need to restart the kernel to use updated packages.



In [22]:
import networkx as nx
import matplotlib.pyplot as plt
from ortools.linear_solver import pywraplp

def coloracao_rodrigues(G):
    # Array de 100 cores fornecido
    color_array = [
        "gold", "red", "violet", "maroon", "limegreen", "greenyellow", "black",
        "blue", "teal", "indigo", "crimson", "turquoise", "coral", "pink",
        "olive", "navy", "magenta", "salmon", "fuchsia", "chartreuse", "skyblue",
        "beige", "sienna", "lavender", "aquamarine", "plum", "steelblue", "khaki",
        "moccasin", "peru", "chocolate", "orchid", "slateblue", "forestgreen",
        "seagreen", "tomato", "turquoise", "mediumvioletred", "cadetblue", "darkcyan",
        "palevioletred", "dodgerblue", "lightseagreen", "lightsalmon", "darkslategray",
        "darkmagenta", "yellowgreen", "mediumpurple", "lightcoral", "springgreen",
        "darkorchid", "powderblue", "rosybrown", "darkgoldenrod", "palegreen", "hotpink",
        "palegoldenrod", "mistyrose", "lightpink", "lime", "midnightblue", "darkseagreen",
        "indianred", "lightgreen", "darkolivegreen", "sandybrown", "mediumslateblue",
        "lightsteelblue", "firebrick", "oceanblue", "lightcyan", "thistle", "lawngreen",
        "royalblue", "seashell", "darkkhaki", "lightskyblue", "navajowhite", "lavenderblush",
        "cornflowerblue", "goldenrod", "darkblue", "darkturquoise", "mediumspringgreen",
        "wheat", "papayawhip", "lightgoldenrodyellow", "peru", "bisque", "peachpuff",
        "lemonchiffon", "lightgray", "mediumorchid", "darkred", "burlywood", "slategray",
        "mediumseagreen", "antiquewhite", "honeydew", "mintcream"
    ]

    vertices = list(G.nodes())
    n = len(vertices)

    # Verificar se o grafo foi corretamente inicializado
    if n == 0:
        raise ValueError("O grafo não foi inicializado corretamente.")

    # Inicializando o solver
    solver = pywraplp.Solver.CreateSolver('SCIP')

    # Verificar se o solver foi corretamente inicializado
    if not solver:
        raise ValueError("O solver não foi inicializado corretamente.")

    # Definir o tempo máximo de execução para o solver (30 minutos)
    solver.SetTimeLimit(1800 * 1000)  # 1800 segundos em milissegundos

    # Variáveis de decisão x[v, c]: 1 se o vértice v recebe a cor c, 0 caso contrário
    x = {}
    num_colors = len(color_array)  # Número de cores é o tamanho do array de cores fornecido
    for v in vertices:
        for c in range(1, num_colors + 1):
            x[v, c] = solver.BoolVar(f'x[{v},{c}]')

    # Variáveis z_c: 1 se a cor c é usada, 0 caso contrário
    z = {}
    for c in range(1, num_colors + 1):
        z[c] = solver.BoolVar(f'z[{c}]')

    # Função objetivo: minimizar o número de cores usadas
    solver.Maximize(solver.Sum(z[c] for c in range(1, num_colors + 1)))

    # 1. Restrição de coloração única: cada vértice recebe exatamente uma cor
    for v in vertices:
        solver.Add(solver.Sum(x[v, c] for c in range(1, num_colors + 1)) == 1)

    # 2. Restrição de coloração própria: vértices adjacentes não podem ter a mesma cor
    for v in vertices:
        for u in G.neighbors(v):  # Para cada vizinho u de v
            if v < u:  # Evitar duplicação de restrições
                for c in range(1, num_colors + 1):
                    solver.Add(x[v, c] + x[u, c] <= z[c])

    # 3. Uma cor só pode ser usada se for atribuída a pelo menos um vértice
    for c in range(1, num_colors + 1):
        solver.Add(z[c] <= solver.Sum(x[v, c] for v in vertices))

    # 4. Propriedade de Grundy
    for v in vertices:
        for c in range(2, num_colors + 1):
            solver.Add(x[v, c] >= 1 - solver.Sum(x[v, d] for d in range(1, c)) - solver.Sum(x[u, c] for u in G.neighbors(v)))

    # 5. As cores devem ser usadas em ordem
    for c in range(2, num_colors + 1):
        solver.Add(z[c] <= z[c - 1])

    # 6. Vértices isolados devem receber a primeira cor
    for v in vertices:
        if len(list(G.neighbors(v))) == 0:
            solver.Add(x[v, 1] == 1)

    # Resolver o problema
    print("Resolvendo o problema...")
    status = solver.Solve()

    # Verificar o status da solução
    color_map = []

    if status == pywraplp.Solver.OPTIMAL:
        print('Solution:')
        print('Objective value =', solver.Objective().Value())

        for v in vertices:
            for c in range(1, num_colors + 1):
                if x[v, c].solution_value() == 1:
                    color_map.append(color_array[c-1])  # Mapeia o índice da cor para o color_map


    elif status == pywraplp.Solver.FEASIBLE:
        print('A feasible solution was found within the time limit.')
        print('Objective value =', solver.Objective().Value())

        for v in vertices:
            for c in range(1, num_colors + 1):
                if x[v, c].solution_value() == 1:
                    color_map.append(color_array[c-1])  # Mapeia o índice da cor para o color_map


    else:
        print('The problem does not have an optimal solution within the time limit.')

    print('Problem solved in %f milliseconds' % (solver.WallTime()/1000));
    print('Problem solved in %d iterations' % solver.iterations());
    print('Problem solved in %d branch-and-bound nodes' % solver.nodes())
    


In [7]:
import networkx as nx
G = nx.gnp_random_graph(10, 0.5)
coloracao_rodrigues(G)


Resolvendo o problema...
Solution:
Objective value = 6.000000000000118
Problem solved in 28.799000 milliseconds
Problem solved in 32604 iterations
Problem solved in 5 branch-and-bound nodes


In [9]:
import networkx as nx
G = nx.gnp_random_graph(10, 0.8)
coloracao_rodrigues(G)


Resolvendo o problema...
Solution:
Objective value = 7.999999999996472
Problem solved in 44.492000 milliseconds
Problem solved in 35684 iterations
Problem solved in 43 branch-and-bound nodes


In [11]:
import networkx as nx
G = nx.gnp_random_graph(13, 0.5)
coloracao_rodrigues(G)


Resolvendo o problema...
Solution:
Objective value = 8.0
Problem solved in 41.122000 milliseconds
Problem solved in 60373 iterations
Problem solved in 911 branch-and-bound nodes


In [13]:
import networkx as nx
G = nx.gnp_random_graph(13, 0.8)
coloracao_rodrigues(G)

Resolvendo o problema...
Solution:
Objective value = 9.0
Problem solved in 413.053000 milliseconds
Problem solved in 341277 iterations
Problem solved in 5853 branch-and-bound nodes


In [19]:
import networkx as nx
G = nx.gnp_random_graph(15, 0.5)
coloracao_rodrigues(G)

Resolvendo o problema...
Solution:
Objective value = 9.0
Problem solved in 57.521000 milliseconds
Problem solved in 204281 iterations
Problem solved in 2299 branch-and-bound nodes


In [17]:
import networkx as nx
G = nx.gnp_random_graph(15, 0.8)
coloracao_rodrigues(G)

Resolvendo o problema...
Solution:
Objective value = 11.0
Problem solved in 562.450000 milliseconds
Problem solved in 2255192 iterations
Problem solved in 61533 branch-and-bound nodes


In [21]:
import networkx as nx
G = nx.gnp_random_graph(18, 0.5)
coloracao_rodrigues(G)

Resolvendo o problema...
Solution:
Objective value = 9.999999999999995
Problem solved in 237.109000 milliseconds
Problem solved in 1064644 iterations
Problem solved in 9522 branch-and-bound nodes


In [23]:
import networkx as nx
G = nx.gnp_random_graph(18, 0.8)
coloracao_rodrigues(G)

Resolvendo o problema...
A feasible solution was found within the time limit.
Objective value = 11.0
Problem solved in 1799.019000 milliseconds
Problem solved in 1612049 iterations
Problem solved in 1 branch-and-bound nodes
